# Chapter 9: Capstone Project - A Modular Agentic System

## Putting It All Together

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sanjaynegi309/agentic-ai-course/blob/main/notebooks/09-Capstone-Building-a-Modular-Agentic-System.ipynb)

Congratulations! You've reached the final chapter and the capstone project. It's time to combine everything you've learned—LangChain, CrewAI, LangGraph, Hugging Face, and the Gemini API—into a single, powerful, and modular agentic system. Let's build something impressive.

## 🚀 The Mission: A Content Creation & Promotion Pipeline

Our goal is to build an automated system that can:
1.  Take a topic.
2.  Use a **CrewAI** team to research the topic and write a blog post.
3.  Use a **native Gemini API** call for a quick review of the post.
4.  If the post needs revisions, loop back to the CrewAI team with feedback (**LangGraph** will manage this loop).
5.  Once approved, use a specialized **Hugging Face** model to generate a promotional tweet for the blog post.

This project demonstrates the power of using the right tool for the right job, orchestrated by a master workflow manager.

## 🛠️ Our Toolkit

| Framework/API | Role in Our System |
|---|---|
| **LangGraph** | The **Master Orchestrator**. It will define the overall workflow, manage the state, and handle the conditional logic (the review loop). |
| **CrewAI** | The **Specialist Team**. It will handle the self-contained, multi-step task of researching and writing the initial draft. We'll use it as a 'black box' node in our graph. |
| **Hugging Face**| The **Niche Specialist**. We'll use a smaller, specialized model for the specific task of generating a short tweet. |
| **Gemini API** | The **Quick Consultant**. We'll use a direct API call for the review step, as it's a simple, single-shot task that doesn't require a full agent setup. |

In [ ]:
# Step 1: The Grand Setup
!pip install langgraph crewai langchain langchain_google_genai huggingface_hub transformers python-dotenv duckduckgo-search torch

import os
from dotenv import load_dotenv
if not load_dotenv():
    try:
        from google.colab import userdata
        os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
        os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API_KEY')
    except ImportError:
        print("Could not load API keys.")

### Step 2: Building the CrewAI Node

First, we'll create our research and writing crew. The key here is to wrap the entire CrewAI process in a single function that we can use as a node in LangGraph.

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun

def create_crewai_node(topic, feedback=None):
    """Creates and runs the CrewAI research and writing crew."""
    search_tool = DuckDuckGoSearchRun()
    
    researcher = Agent(
        role='Expert AI Researcher',
        goal=f'Find the most relevant and up-to-date information on {topic}.',
        backstory='You are a meticulous researcher, skilled at finding facts and synthesizing them.',
        tools=[search_tool],
        allow_delegation=False
    )
    
    writer = Agent(
        role='Professional Tech Blogger',
        goal=f'Write an engaging and informative blog post on {topic}.',
        backstory='You are a skilled writer, able to turn complex topics into clear, compelling narratives.',
        allow_delegation=False
    )
    
    research_task = Task(
        description=f'Research the topic: {topic}.',
        expected_output='A detailed summary of key points, trends, and examples.',
        agent=researcher
    )
    
    writing_description = f'Write a blog post on {topic}.'
    if feedback:
        writing_description += f' Please incorporate the following feedback: {feedback}'
        
    writing_task = Task(
        description=writing_description,
        expected_output='A well-structured, 4-paragraph blog post.',
        agent=writer
    )
    
    crew = Crew(agents=[researcher, writer], tasks=[research_task, writing_task], process=Process.sequential)
    return crew.kickoff()

### Step 3: The Hugging Face & Gemini Nodes

Next, we'll create functions for our other specialists: the reviewer (using the native Gemini API) and the social media manager (using a Hugging Face model).

In [ ]:
import google.generativeai as genai
from langchain_huggingface import HuggingFacePipeline

# Gemini node for reviewing
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
reviewer_llm = genai.GenerativeModel('gemini-pro')

def reviewer_node_func(blog_post):
    prompt = f"Review the following blog post. If it is high quality and ready to publish, respond with 'APPROVED'. Otherwise, provide concise feedback for improvement.\n\n{blog_post}"
    response = reviewer_llm.generate_content(prompt)
    return response.text

# Hugging Face node for tweeting
tweet_generator_llm = HuggingFacePipeline.from_model_id(
    model_id="distilgpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 280, "pad_token_id": 50256},
)

def tweet_generator_node_func(blog_post):
    prompt = f"Create a short, punchy tweet to promote this blog post: {blog_post[:500]}"
    response = tweet_generator_llm.invoke(prompt)
    return response

### Step 4: The LangGraph Orchestrator

Now, let's wire everything together using LangGraph.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# Define the state
class CapstoneState(TypedDict):
    topic: str
    blog_post: str
    review_feedback: str
    tweet: str

# Define the graph nodes
def content_creation_node(state):
    print("--- Node: Content Creation Crew ---")
    post = create_crewai_node(state['topic'], state.get('review_feedback'))
    return {"blog_post": post, "review_feedback": None} # Clear feedback after revision

def review_node(state):
    print("--- Node: Reviewer ---")
    feedback = reviewer_node_func(state['blog_post'])
    return {"review_feedback": feedback}

def social_media_node(state):
    print("--- Node: Social Media Specialist ---")
    tweet = tweet_generator_node_func(state['blog_post'])
    return {"tweet": tweet}

# Define the conditional edge
def review_decision(state):
    if "APPROVED" in state['review_feedback']:
        print("--- Decision: Approved ---")
        return "social_media"
    else:
        print("--- Decision: Revision Needed ---")
        return "content_creation"

# Assemble the graph
graph = StateGraph(CapstoneState)
graph.add_node("content_creation", content_creation_node)
graph.add_node("review", review_node)
graph.add_node("social_media", social_media_node)

graph.set_entry_point("content_creation")
graph.add_edge("content_creation", "review")
graph.add_conditional_edges("review", review_decision, {"social_media": "social_media", "content_creation": "content_creation"})
graph.add_edge("social_media", END)

runnable_graph = graph.compile()

In [ ]:
# Step 5: Run the Pipeline!
inputs = {"topic": "The impact of generative AI on creative industries"}
final_state = runnable_graph.invoke(inputs)

print("\n\n--- FINAL OUTPUT ---")
print("\n**Generated Blog Post:**")
print(final_state['blog_post'])
print("\n**Generated Tweet:**")
print(final_state['tweet'])

## 🎉 Course Conclusion

Congratulations on building a truly sophisticated, multi-framework agentic system! You have successfully orchestrated a team of specialized agents, managed a complex workflow with loops, and integrated both proprietary and open-source models.

You are no longer just a user of AI; you are an architect of intelligent systems. You have the skills to design and build modular, powerful, and efficient AI agents for a vast range of applications.

The journey doesn't end here. The field of agentic AI is one of the most exciting and rapidly evolving areas of technology. Continue to experiment, build your own projects, and share your work with the community.

**Thank you for taking this course. Now go build the future!**